In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

In [ ]:
# default_exp core

In [ ]:
#export
from fastai.tabular.all import *

In [ ]:
#hide
from nbdev.showdoc import *

# Time Series Feature Engineering Core

> Basic functions for time series analysis.

In [ ]:
# from fastai.tabular.all import *
# https://www.analyticsvidhya.com/blog/2019/12/6-powerful-feature-engineering-techniques-time-series/

def add_lag_features(df, field_name, prefix=None, lag_periods=[1]):
    "Helper function that adds lag features relevant to the column `field_name` of `df`."
    field = df[field_name]
    prefix = ifnone(prefix, field_name)
    for n in lag_periods: df[f'{prefix}-{n}p'] = df[field_name].shift(n)
    return df

def add_lag_percentage_gain_features(df, field_name, prefix=None, lag_periods=[1]):
    "Helper function that adds lag percentage gain features relevant to the column `field_name` of `df`."
    field = df[field_name]
    prefix = ifnone(prefix, field_name)
    for n in lag_periods:
        df[f'{prefix}-{n}p_PG'] = df[field_name]/df[field_name].shift(n)
    return df

def add_moving_average_features(df, field_name, prefix=None, windows=[3], weighted=True):
    "Helper function that adds moving average (rolling window) features relevant to the column `field_name` of `df`."
    field = df[field_name]
    prefix = ifnone(prefix, field_name)
    for n in windows:
        if weighted:
            weights = np.arange(1, n + 1)
            df[f'{prefix}_{n}p_MA'] = df[field_name].rolling(
                window=n).apply(lambda x: np.dot(x, weights) /
                                       weights.sum(), raw=True)
        else:
            df[f'{prefix}_{n}p_MA'] = df[field_name].rolling(window=n).mean()
    return df

def add_moving_average_percentage_gain_features(df, field_name, prefix=None, windows=[3], weighted=True):
    "Helper function that adds moving average (rolling window) percentage gain features relevant to the column `field_name` of `df`."
    field = df[field_name]
    prefix = ifnone(prefix, field_name)
    for n in windows:
        if weighted:
            weights = np.arange(1, n + 1)
            df[f'{prefix}_{n}p_MA_PG'] = df[field_name]/df[field_name].rolling(
                window=n).apply(lambda x: np.dot(x, weights) /
                                       weights.sum(), raw=True)
        else:
            df[f'{prefix}_{n}p_MA_PG'] = df[field_name]/df[field_name].rolling(window=n).mean()
    return df

def add_expanding_features(df, field_name, prefix=None, period=7):
    "Helper function that adds expanding features relevant to the column `field_name` of `df`."
    field = df[field_name]
    prefix = ifnone(prefix, field_name)
    for n in windows:
        df[f'{prefix}_{n}p_expanding'] = df[field_name].expanding(n).mean()
    return df

def add_trend_features(df, field_name, prefix=None, windows=[3]):
    "Helper function that adds trend features relevant to the column `field_name` of `df`."
    field = df[field_name]
    prefix = ifnone(prefix, field_name)
    for n in windows:
        df[f'{prefix}_{n}p_trend'] = (df[field_name]
                .rolling(window=n)
                .mean()
                .diff()
                .fillna(0))
    return df